# Quantum Demonstration Playground

This notebook will be used for demonstrations of simple gate computations. Import statements and some basic stubs are provided.

In [ ]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

In [ ]:
q = np.array([[1], [0]])
q

In [ ]:
# Gate definitions
i = np.array([[1, 0], [0, 1]])
x = np.array([[0, 1], [1, 0]])
h = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
i, x, h

# Demonstrations

In [ ]:
circuit = QuantumCircuit(1)

circuit.i(0)
# circuit.x(0)
# circuit.h(0)

circuit.draw('mpl')

In [ ]:
state = Statevector(q)
state.draw('latex')

In [ ]:
state = state.evolve(circuit)
state.draw('latex')